BDLE 2022

date du document  :  16/12/2022

# TP 9 et 10 : traitement sur des partitions : tri, regroupement, fenetre, jointure
Version ETUDIANT



## Préparation

Pour accéder directement aux fichiers stockées sur votre google drive. Renseigner le code d'authentification lorsqu'il est demandé

Ajuster le nom de votre dossier : MyDrive/essai

In [2]:
# import os
# from google.colab import drive
# drive.mount("/content/drive")

# drive_dir = "/content/drive/MyDrive/essai"
# os.makedirs(drive_dir, exist_ok=True)
# os.listdir(drive_dir)

Installer pyspark et findspark :


In [3]:
!pip install -q pyspark
!pip install -q findspark
print("installé")

     |████████████████████████████████| 281.4 MB 37 kB/s 
     |████████████████████████████████| 199 kB 58.7 MB/s 
installé


Démarrer la session spark

In [4]:
import os
# !find /usr/local/lib/ -name "pyspark" 
os.environ["SPARK_HOME"] = "/usr/local/lib/python3.8/dist-packages/pyspark"
os.environ["JAVA_HOME"] = "/usr"

In [5]:
# Principaux import
import findspark
from pyspark.sql import SparkSession 
from pyspark import SparkConf  

# pour les dataframe et udf
from pyspark.sql import *  
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import *

# pour le chronomètre
import time

# initialise les variables d'environnement pour spark
findspark.init()

# Démarrage session spark 
# --------------------------
def demarrer_spark():
  local = "local[*]"
  appName = "TP"
  configLocale = SparkConf().setAppName(appName).setMaster(local).\
  set("spark.executor.memory", "6G").\
  set("spark.driver.memory","3G").\
  set("spark.sql.catalogImplementation","in-memory")
  
  spark = SparkSession.builder.config(conf = configLocale).getOrCreate()
  sc = spark.sparkContext
  sc.setLogLevel("ERROR")
  
  spark.conf.set("spark.sql.autoBroadcastJoinThreshold","-1")

  # On ajuste l'environnement d'exécution des requêtes à la taille du cluster (4 coeurs)
  spark.conf.set("spark.sql.shuffle.partitions","4")    
  print("session démarrée, son id est ", sc.applicationId)
  return spark
spark = demarrer_spark()

session démarrée, son id est  local-1672756012689


Redéfinir la fonction **display** pour afficher le resutltat des requêtes dans un tableau

In [7]:
import pandas as pd
from google.colab import data_table

# alternatives to Databricks display function.

def display(df, n=100):
  return data_table.DataTable(df.limit(n).toPandas(), include_index=False, num_rows_per_page=10)

def display2(df, n=20):
  pd.set_option('max_columns', None)
  pd.set_option('max_colwidth', None)
  return df.limit(n).toPandas().head(n)

Définir le tag **%%sql** pour pouvoir écrire plus simplement des requêtes en SQL dans une cellule

In [8]:
from IPython.core.magic import (register_line_magic, register_cell_magic, register_line_cell_magic)

def removeComments(query):
  result = ""
  for line in query.split('\n'):
    if not(line.strip().startswith("--")):
      result += line + "\n"
  return result

@register_line_cell_magic
def sql(line, cell=None):
    "To run a sql query. Use:  %%sql"
    val = cell if cell is not None else line
    tabRequetes = removeComments(val).split(";")
    derniere = None
    est_requete = False
    for r in tabRequetes:
        r = r.strip()
        if len(r) > 2:
          derniere = spark.sql(r)
          est_requete = ( r.lower().startswith('select') or r.lower().startswith('with') )
    if(est_requete):
      return display(derniere)
    else:
      return print('ok')

Utiliaires : Chronomètres

In [9]:
#------------------------------
# Chronometre : chronoPersist2
#------------------------------
import time

# Ce chronometre garantit que chaque tuple du dataframe est lu entièrement.
# En effet il est nécessaire de lire le détail de chaque tuple avant de les 'copier' en mémoire.
def chronoPersist(df):
    df.unpersist()
    t1 = time.perf_counter()
    count = df.persist().count()
    t2 = time.perf_counter()
    df.unpersist()
    print('durée: {:.1f} s'.format(t2 - t1), 'pour lire', count , 'elements')

def chronoPersist2(df):
  dest = df.selectExpr("*", "1")
  t1 = time.perf_counter()
  count = dest.persist().count()
  t2 = time.perf_counter()
  dest.unpersist()
  print('durée: {:.1f} s'.format(t2 - t1), 'pour lire', count , 'elements')
        
def chronoCount(df):
  t1 = time.perf_counter()
  count = df.count()
  t2 = time.perf_counter()
  print('durée: {:.1f} s'.format(t2 - t1), 'pour dénombrer', count , 'elements')
    
print("fonctions définies")

fonctions définies


## Accès aux données

In [10]:
import os
local_dir = "/local/data"
os.makedirs(local_dir, exist_ok=True)
os.listdir(local_dir)

[]

URL pour l'accès aux datasets

In [11]:
# ---------------------------------------------------------------------------
# en cas de problème avec le téléchargement des datasets, aller directement sur l'URL ci-dessous
PUBLIC_DATASET_URL = "https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4" 
PUBLIC_DATASET=PUBLIC_DATASET_URL + "/download?path="

print("URL pour les datasets ", PUBLIC_DATASET_URL)

URL pour les datasets  https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4


In [12]:
%cd /local/data
! unzip notes1M.zip

/local/data
unzip:  cannot find or open notes1M.zip, notes1M.zip.zip or notes1M.zip.ZIP.


In [13]:
import os
from urllib import request
import zipfile

# download dataset if not already donwloaded
def download_file(web_dir, local_dir, file):
  local_file = local_dir + "/" + file
  web_file = web_dir + "/" + file
  if(os.path.isfile(local_file)):
    print(file, "is already stored")
  else:
    print("downloading from URL: ", web_file , "save in : " + local_file)
    request.urlretrieve(web_file , local_file)

def unzip_file(local_dir, file):
  with zipfile.ZipFile(local_dir + "/" + file, 'r') as zip_ref:
    zip_ref.extractall(local_dir)
  # os.remove(local_dir + "/" + file)


web_dir = PUBLIC_DATASET + "/movielens"

download_file(web_dir, local_dir, "notes1M.zip")
unzip_file(local_dir, "notes1M.zip")

download_file(web_dir, local_dir, "ratings3M.zip")
unzip_file(local_dir, "ratings3M.zip")

download_file(web_dir, local_dir, "films.json")

web_dir = PUBLIC_DATASET + "/movielens/ml-latest"

download_file(web_dir, local_dir, "movies.csv")


# Liste des fichiers
os.listdir(local_dir)

downloading from URL:  https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4/download?path=/movielens/notes1M.zip save in : /local/data/notes1M.zip
downloading from URL:  https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4/download?path=/movielens/ratings3M.zip save in : /local/data/ratings3M.zip
downloading from URL:  https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4/download?path=/movielens/films.json save in : /local/data/films.json
downloading from URL:  https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4/download?path=/movielens/ml-latest/movies.csv save in : /local/data/movies.csv


['ratings3M.zip',
 'movies.csv',
 'notes1M.zip',
 'notes1M.json',
 'films.json',
 'ratings3M.csv']

### Dataframe les Films

In [14]:
# schema_film = "nf long, titre String, g Array<String>"
schema_film = StructType([StructField('nF',LongType()),
                          StructField('titre',StringType()),
                          StructField('g',ArrayType(StringType()))])

films = spark.read.json(local_dir + "/" + "films.json", schema = schema_film).selectExpr("nF", "titre", "g as genres")
# print('schema:', films.schema)

# extrait
film_extrait = films.where("nF <100").repartition(3).persist()

film_extrait.printSchema()
display(film_extrait)

root
 |-- nF: long (nullable = true)
 |-- titre: string (nullable = true)
 |-- genres: array (nullable = true)
 |    |-- element: string (containsNull = true)



,nF,titre,genres
0,94,Beautiful Girls (1996),"[Comedy, Drama, Romance]"
1,45,To Die For (1995),"[Comedy, Drama, Thriller]"
2,58,"Postman, The (Postino, Il) (1994)","[Comedy, Drama, Romance]"
3,54,"Big Green, The (1995)","[Children, Comedy]"
4,9,Sudden Death (1995),[Action]
...,...,...,...
87,38,It Takes Two (1995),"[Children, Comedy]"
88,15,Cutthroat Island (1995),"[Action, Adventure, Romance]"
89,85,Angels and Insects (1995),"[Drama, Romance]"
90,64,Two if by Sea (1996),"[Comedy, Romance]"


In [15]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Dataframe Les Notes

In [16]:
notes_schema = StructType([StructField('nF',LongType()),
                           StructField('nU',LongType()),
                           StructField('note',DoubleType()),
                           StructField('annee',LongType())])

notes = spark.read.json(local_dir + "/" + "notes1M.json", schema = notes_schema).selectExpr("nF", "nU", "note", "annee")

#extrait
notes_extrait = notes.where("nU < 1000").join(film_extrait, "nF").select(notes["nF"], "nU", "note", "annee").repartition(3).persist()
print(notes_extrait.count())
display(notes_extrait)

185


,nF,nU,note,annee
0,19,76,1.5,2011
1,62,691,5.0,1996
2,25,832,5.0,1998
3,5,296,3.5,2005
4,48,781,3.0,1996
...,...,...,...,...
95,69,798,3.0,2000
96,95,430,3.0,1996
97,10,984,4.0,1996
98,36,261,2.0,1997


# Exercice 1 : Traitement sur des partitions

#### Fonction showPartitionSize

La fonction *showPartitionSize*  affiche le nombre d'éléments dans chaque partition

In [17]:
# La fonction part_size prend en paramètres un numéro de partition (partID) et un itérateur sur une partition.
# Elle retourne un itérateur sur une partition qui contient un seul tuple (num_partition, nbtuples).

def partSize(partID, iterateur):
  size=0
  suivant = next(iterateur, None)
  while suivant is not None :
    size += 1
    suivant = next(iterateur, None)
  yield (partID, size)


def showPartitionSize(df):
  if df.isEmpty():
    print("empty dataframe")
  else:
    #invoquer la fonction partSize sur chaque partition 
    t = df.selectExpr("1").rdd.mapPartitionsWithIndex(partSize).toDF(['partID', 'size'])

    # Rmq : selectExpr("1") sert à simplifier la partition pour ne garder qu'une seule colonne contentant des "1",
    # ce qui est suffisant pour compter le nombre de tuples dans la partition.

    #affichage
    t.show()

print('showPartitionSize définie')

showPartitionSize définie


#### Fonction showPartitions
 
La fonction *showPartitions*  affiche les _n_ premiers éléments de chaque partition


In [18]:
def showPartitions(df, N=5 , display_per_partition=True):
  if df.isEmpty():
     print("empty dataframe")
  else:   
    nb_partitions = df.rdd.getNumPartitions()

    # fonction topN est invoquée sur une partition.
    # topN retourne un itérateur sur une partition qui contient les N premiers éléments de cette partition
    def topN(partID, iterateur):
      c=0
      suivant = next(iterateur, None)
      while suivant is not None and c < N :
        c+=1
        tuple_avec_numero_partition = (partID, *suivant)
        suivant = next(iterateur, None)
        yield tuple_avec_numero_partition
    #-- fin de topN ---

    partid_tmp = "_p_"
    nom_attributs = [partid_tmp] + df.schema.fieldNames()
    premiers_tuples = df.rdd.mapPartitionsWithIndex(topN).toDF(nom_attributs)
    premiers_tuples.persist()

    if(display_per_partition):
      # afficher séparément le contenu du début de chaque partition
      for partID in range(nb_partitions):
        print("partition", partID)
        # afficher la partition partID
        premiers_tuples.where(col(partid_tmp)==partID).drop(partid_tmp).show(N, False)
    else:
      return display(premiers_tuples)
   
print('showPartitions définie')

showPartitions définie


Afficher la taille de chaque partition

In [19]:
showPartitionSize(film_extrait)

+------+----+
|partID|size|
+------+----+
|     0|  31|
|     1|  31|
|     2|  30|
+------+----+



Afficher les premiers éléments de chaque partition

In [20]:
showPartitions(film_extrait,5)

partition 0
+---+---------------------------------+-------------------------+
|nF |titre                            |genres                   |
+---+---------------------------------+-------------------------+
|94 |Beautiful Girls (1996)           |[Comedy, Drama, Romance] |
|45 |To Die For (1995)                |[Comedy, Drama, Thriller]|
|58 |Postman, The (Postino, Il) (1994)|[Comedy, Drama, Romance] |
|54 |Big Green, The (1995)            |[Children, Comedy]       |
|9  |Sudden Death (1995)              |[Action]                 |
+---+---------------------------------+-------------------------+

partition 1
+---+--------------------------------------------+-----------------+
|nF |titre                                       |genres           |
+---+--------------------------------------------+-----------------+
|14 |Nixon (1995)                                |[Drama]          |
|71 |Fair Game (1995)                            |[Action]         |
|49 |When Night Is Falling (1995)   

## Question 1 : Tri des films par titre

En vous inspirant de la fonction *topN*  définir une fonction qui tri les données de *film_extrait* par titre. Le résultat a le meme schéma que film.

In [21]:

def tri_par_titre(iterateur):
  suivant = next(iterateur, None)
  groupes = [[] for x in range(3)]
  while suivant is not None:
    lettre1 = suivant.titre[0]
    if lettre1 < 'H':
      p=0
    elif lettre1 < 'P':
      p=1
    else:
      p=2
    groupes[p].append((p, *suivant))
    suivant = next(iterateur, None)
  for groupe in groupes:
    for film in sorted(groupe, key=lambda t: t[2]):
      yield film

# tester la fonction
# it = iter(film_extrait.limit(3).collect())
# for x in tri_par_titre(it):
#   print(x)

names = ["p"] + film_extrait.schema.fieldNames()
film1 = film_extrait.rdd.mapPartitions(tri_par_titre).toDF(names)
showPartitions(film1,30)

partition 0
+---+---+----------------------------------------------------+-------------------------------------------------+
|p  |nF |titre                                               |genres                                           |
+---+---+----------------------------------------------------+-------------------------------------------------+
|0  |13 |Balto (1995)                                        |[Adventure, Animation, Children]                 |
|0  |94 |Beautiful Girls (1996)                              |[Comedy, Drama, Romance]                         |
|0  |54 |Big Green, The (1995)                               |[Children, Comedy]                               |
|0  |88 |Black Sheep (1996)                                  |[Comedy]                                         |
|0  |35 |Carrington (1995)                                   |[Drama, Romance]                                 |
|0  |39 |Clueless (1995)                                     |[Comedy, Romance]     

repartitionner les données par **intervalle** en fonction de l'attribut p

In [22]:
film2 = film1.repartitionByRange(3, col('p'))
showPartitions(film2,20)

partition 0
+---+---+--------------------------------------------+-----------------------------------------+
|p  |nF |titre                                       |genres                                   |
+---+---+--------------------------------------------+-----------------------------------------+
|0  |13 |Balto (1995)                                |[Adventure, Animation, Children]         |
|0  |94 |Beautiful Girls (1996)                      |[Comedy, Drama, Romance]                 |
|0  |54 |Big Green, The (1995)                       |[Children, Comedy]                       |
|0  |88 |Black Sheep (1996)                          |[Comedy]                                 |
|0  |35 |Carrington (1995)                           |[Drama, Romance]                         |
|0  |39 |Clueless (1995)                             |[Comedy, Romance]                        |
|0  |22 |Copycat (1995)                              |[Crime, Drama, Horror, Mystery, Thriller]|
|0  |36 |Dead Man 

In [23]:
film_tri_total = film2.sortWithinPartitions('titre')
showPartitions(film_tri_total,15)

partition 0
+---+---+-------------------------------------+--------------------------------+
|p  |nF |titre                                |genres                          |
+---+---+-------------------------------------+--------------------------------+
|0  |19 |Ace Ventura: When Nature Calls (1995)|[Comedy]                        |
|0  |37 |Across the Sea of Time (1995)        |[Documentary, IMAX]             |
|0  |11 |American President, The (1995)       |[Comedy, Drama, Romance]        |
|0  |85 |Angels and Insects (1995)            |[Drama, Romance]                |
|0  |82 |Antonia's Line (Antonia) (1995)      |[Comedy, Drama]                 |
|0  |23 |Assassins (1995)                     |[Action, Crime, Thriller]       |
|0  |34 |Babe (1995)                          |[Children, Drama]               |
|0  |13 |Balto (1995)                         |[Adventure, Animation, Children]|
|0  |94 |Beautiful Girls (1996)               |[Comedy, Drama, Romance]        |
|0  |74 |Bed of 

## Question 2 : Regrouper les films par genre et les compter

En vous inspirant de la fonction *topN*  définir une fonction qui compte le nombre de films par genre. Le schéma du résultat est (genre, n)

In [32]:
def count_par_genre(films):
  dic = {}
  for film in films:
    genres = film.genres
    for genre in genres:
      c = dic.get(genre, 0)
      dic[genre] = c + 1
  for genre, count in dic.items():
    yield genre, count

films_par_genre = film_extrait.rdd.mapPartitions(count_par_genre).toDF()

showPartitions(films_par_genre, 15)

partition 0
+---------+---+
|_1       |_2 |
+---------+---+
|Comedy   |13 |
|Drama    |19 |
|Romance  |10 |
|Thriller |5  |
|Children |6  |
|Action   |5  |
|Crime    |5  |
|War      |1  |
|Horror   |4  |
|Adventure|4  |
|Animation|2  |
|Sci-Fi   |2  |
|Fantasy  |2  |
|Mystery  |1  |
+---------+---+

partition 1
+-----------+---+
|_1         |_2 |
+-----------+---+
|Drama      |17 |
|Action     |8  |
|Romance    |7  |
|Children   |4  |
|Mystery    |2  |
|Comedy     |7  |
|Documentary|1  |
|Crime      |5  |
|Thriller   |8  |
|Adventure  |2  |
|Fantasy    |1  |
|Sci-Fi     |1  |
|Animation  |1  |
|Musical    |1  |
+-----------+---+

partition 2
+-----------+---+
|_1         |_2 |
+-----------+---+
|Comedy     |11 |
|Drama      |14 |
|Crime      |4  |
|Thriller   |7  |
|Action     |4  |
|Adventure  |6  |
|Children   |2  |
|Mystery    |3  |
|Horror     |1  |
|Documentary|2  |
|Romance    |6  |
|War        |1  |
|Fantasy    |2  |
|Sci-Fi     |2  |
|IMAX       |1  |
+-----------+---+



In [29]:
#code pour afficher le nombre total de films par genre dans l'ensemble de données

df = films.toPandas().explode('genres').copy().drop_duplicates().groupby(['genres'])['genres'].count()
df

genres
(no genres listed)      18
Action                1545
Adventure             1117
Animation              447
Children               583
Comedy                3315
Crime                 1100
Documentary            495
Drama                 4365
Fantasy                654
Film-Noir              133
Horror                 877
IMAX                   153
Musical                394
Mystery                543
Romance               1545
Sci-Fi                 792
Thriller              1729
War                    367
Western                168
Name: genres, dtype: int64

## Question 3 : Numérotation

En vous inspirant de la fonction *topN*  définir une fonction qui attribue un numéro de 1 à n à chaque tuple de notes_extrait. Le résultat a les attributs de notes + un attribut num.

In [33]:
def numeroter(notes):
  for i, note in enumerate(notes):
    yield (i+1, *note)

nom_col = ["num"] + notes_extrait.schema.fieldNames() 
notes_avec_num = notes_extrait.rdd.mapPartitions(numeroter).toDF(nom_col)

showPartitions(notes_avec_num, 15)

partition 0
+---+---+---+----+-----+
|num|nF |nU |note|annee|
+---+---+---+----+-----+
|1  |19 |76 |1.5 |2011 |
|2  |62 |691|5.0 |1996 |
|3  |25 |832|5.0 |1998 |
|4  |5  |296|3.5 |2005 |
|5  |48 |781|3.0 |1996 |
|6  |34 |871|4.0 |2000 |
|7  |2  |796|1.5 |2015 |
|8  |2  |353|3.0 |2003 |
|9  |28 |815|5.0 |1996 |
|10 |5  |196|4.0 |1996 |
|11 |62 |760|4.0 |1996 |
|12 |29 |882|5.0 |1996 |
|13 |62 |989|3.0 |1997 |
|14 |12 |459|3.0 |2015 |
|15 |62 |87 |3.0 |1997 |
+---+---+---+----+-----+

partition 1
+---+---+---+----+-----+
|num|nF |nU |note|annee|
+---+---+---+----+-----+
|1  |15 |875|4.0 |1996 |
|2  |62 |368|4.0 |2006 |
|3  |29 |592|4.0 |2000 |
|4  |31 |767|3.0 |1996 |
|5  |25 |339|5.0 |1997 |
|6  |34 |682|4.0 |2000 |
|7  |19 |63 |0.5 |2007 |
|8  |19 |240|1.5 |2003 |
|9  |2  |390|2.0 |2005 |
|10 |19 |533|3.0 |2000 |
|11 |34 |332|2.5 |2009 |
|12 |29 |924|4.0 |2015 |
|13 |34 |881|4.0 |2006 |
|14 |34 |546|5.0 |1996 |
|15 |25 |638|4.0 |1996 |
+---+---+---+----+-----+

partition 2
+---+---+---

## Question 4 : Regrouper et trier

Définir une fonction qui affiche le nombre de notes par année dans l'ordre croissant des années

In [37]:
def compte_notes(ratings):
  counts = {}

  for rating in ratings:
    year = rating.annee
    if year in counts:
      counts[year] += 1
    else:
      counts[year] = 1

  #Ordre croissant des années 
  counts = sorted(counts.items(), key=lambda x: x[0])

  for year, count in counts:
    yield year, count

notes_par_annee = notes_extrait.rdd.mapPartitions(compte_notes).toDF()
notes_par_annee = notes_par_annee.withColumnRenamed("_1", "annee")\
          .withColumnRenamed("_2", "count")

showPartitions(notes_par_annee, 15)


partition 0
+-----+-----+
|annee|count|
+-----+-----+
|1996 |17   |
|1997 |6    |
|1998 |3    |
|1999 |6    |
|2000 |3    |
|2001 |4    |
|2002 |2    |
|2003 |5    |
|2004 |1    |
|2005 |1    |
|2006 |2    |
|2007 |1    |
|2008 |1    |
|2011 |1    |
|2012 |1    |
+-----+-----+

partition 1
+-----+-----+
|annee|count|
+-----+-----+
|1996 |22   |
|1997 |6    |
|1999 |1    |
|2000 |6    |
|2001 |1    |
|2002 |1    |
|2003 |2    |
|2004 |3    |
|2005 |2    |
|2006 |3    |
|2007 |3    |
|2009 |3    |
|2010 |2    |
|2012 |1    |
|2014 |1    |
+-----+-----+

partition 2
+-----+-----+
|annee|count|
+-----+-----+
|1996 |13   |
|1997 |8    |
|1999 |5    |
|2000 |5    |
|2001 |6    |
|2002 |3    |
|2003 |2    |
|2004 |3    |
|2005 |1    |
|2006 |4    |
|2007 |3    |
|2009 |2    |
|2010 |1    |
|2014 |2    |
|2015 |1    |
+-----+-----+

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[annee#92L], functions=[count(1)])
   +- Exchange hashpartitioning(annee#92L, 4), ENS

In [40]:
#code pour afficher le nombre total de notes par année dans l'ensemble de données

q4 = notes.groupBy("annee").agg(count("*")).orderBy("annee")
q4.show()
q4.explain()

+-----+--------+
|annee|count(1)|
+-----+--------+
| 1996|   86449|
| 1997|   38310|
| 1998|   16506|
| 1999|   61508|
| 2000|  101580|
| 2001|   62315|
| 2002|   45606|
| 2003|   54165|
| 2004|   60046|
| 2005|   92707|
| 2006|   60595|
| 2007|   54775|
| 2008|   60693|
| 2009|   50128|
| 2010|   48991|
| 2011|   41492|
| 2012|   39625|
| 2013|   31631|
| 2014|   29274|
| 2015|   95630|
+-----+--------+
only showing top 20 rows

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Sort [annee#92L ASC NULLS FIRST], true, 0
   +- Exchange rangepartitioning(annee#92L ASC NULLS FIRST, 4), ENSURE_REQUIREMENTS, [plan_id=1280]
      +- HashAggregate(keys=[annee#92L], functions=[count(1)])
         +- Exchange hashpartitioning(annee#92L, 4), ENSURE_REQUIREMENTS, [plan_id=1277]
            +- HashAggregate(keys=[annee#92L], functions=[partial_count(1)])
               +- FileScan json [annee#92L] Batched: false, DataFilters: [], Format: JSON, Location: InMemoryFileIndex(1 paths)[file:/lo

## Question 5 : top fréquence

Définir une fonction qui affiche les 5 mots les plus fréquents dans les titres

In [41]:
def top_frequence(films):
  # Tokenize the titles into words
  nbr_mots = {}

  for film in films:
    for i, mot in enumerate(film.titre.split(" ")):
      if i == len(film.titre.split(" ")) - 1:
        continue
      
      if mot in nbr_mots:
        nbr_mots[mot] += 1
      else:
        nbr_mots[mot] = 1
  
  top_frequent = sorted(nbr_mots.items(), key=lambda x: x[1], reverse=True)[:5]
  # Yield the 5 most common words
  for mot, count in top_frequent:
    yield mot, count

names5 = ["word", "count"]
film5 = film_extrait.rdd.mapPartitions(top_frequence).toDF(names5)

showPartitions(film5, 15)

partition 0
+-----+-----+
|word |count|
+-----+-----+
|The  |3    |
|to   |2    |
|White|2    |
|a    |2    |
|Dead |2    |
+-----+-----+

partition 1
+----+-----+
|word|count|
+----+-----+
|the |4    |
|When|3    |
|The |3    |
|in  |3    |
|of  |3    |
+----+-----+

partition 2
+----+-----+
|word|count|
+----+-----+
|The |4    |
|and |3    |
|the |3    |
|of  |2    |
|Two |2    |
+----+-----+



## Question 6 : Jointure

Définir les fonctions pour calculer la jointure entre les extraits de films et de notes

In [47]:
# def jointure_film_notes

q6 = notes.join(films, films.nF == notes.nF)
q6.explain()
print(q6)
q6.show()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- SortMergeJoin [nF#89L], [nF#0L], Inner
   :- Sort [nF#89L ASC NULLS FIRST], false, 0
   :  +- Exchange hashpartitioning(nF#89L, 4), ENSURE_REQUIREMENTS, [plan_id=1598]
   :     +- Filter isnotnull(nF#89L)
   :        +- FileScan json [nF#89L,nU#90L,note#91,annee#92L] Batched: false, DataFilters: [isnotnull(nF#89L)], Format: JSON, Location: InMemoryFileIndex(1 paths)[file:/local/data/notes1M.json], PartitionFilters: [], PushedFilters: [IsNotNull(nF)], ReadSchema: struct<nF:bigint,nU:bigint,note:double,annee:bigint>
   +- Sort [nF#0L ASC NULLS FIRST], false, 0
      +- Exchange hashpartitioning(nF#0L, 4), ENSURE_REQUIREMENTS, [plan_id=1599]
         +- Project [nF#0L, titre#1, g#2 AS genres#6]
            +- Filter isnotnull(nF#0L)
               +- FileScan json [nF#0L,titre#1,g#2] Batched: false, DataFilters: [isnotnull(nF#0L)], Format: JSON, Location: InMemoryFileIndex(1 paths)[file:/local/data/films.json], PartitionFilters: [